## Migrate Enterprise Users
This script allows a user to login to change the UPN Name of an Enteprise Account

#### First, let's import the ArcGIS Python API & Other Required Python Modules

In [ ]:
from arcgis.gis import *

#### Now, let's define some input parameters
A box will appear allowing you to enter your password. When running interactively, you will be prompted to enter a password. Hit the Enter key to continue.

In [ ]:
# Enter the URL to your ArcGIS Online Organization or Portal for ArcGIS
portalURL = 'https://myorg.maps.arcgis.com'
OrgBaseURL = portalURL.split('/')[-1].split('.')[0]
# Enter the username & password for an Administrator in your Organization
username = ''
password = ''

#### Define the Old UPN and New UPN for the user in your Org

*Note*: THIS MUST MATCH THE SAML NAMEID RESPONSE

In [ ]:
orig_UPN = "123456@company.com"
new_UPN = "John.Doe@company.com"
# Below we will automatically get the proper User ID structure
orig_userid = '{}_{}'.format(orig_UPN,OrgBaseURL)
new_userid = '{}_{}'.format(new_UPN,OrgBaseURL)

#### Next, let's find out how many users are in our Organization
*Note*: the default user search is for 100 users, we are overriding that to 1000 users below

In [ ]:
gis = GIS(portalURL, username, password)
allUsers = gis.users.search(max_users=1000)
print('Total Portal Users: ' + str(len(allUsers)))

#### Get the details for the Original User

In [ ]:
olduser = gis.users.get(orig_userid)
olduser

#### Remove ArcGIS Pro Entitlements

In [ ]:
licenses = gis.admin.license
if licenses.get(name="arcgis pro").user_entitlement(username=orig_userid):
    oldUserEntitlements = licenses.get(name="arcgis pro").user_entitlement(username=orig_userid)['entitlements']
    licenses.get(name="arcgis pro").revoke(username=orig_userid, entitlements=oldUserEntitlements, supress_email=True)
    print('Removed Entitlements')
else:
    print('No entitlements to Remove')
if bool(licenses.get(name="arcgis pro").user_entitlement(username=orig_userid)) == False:
    print('User has no entitlements')

#### Create a New User Based on the Old User's Info

In [ ]:
if olduser.role == 'org_admin':
    # Based on the old user's info, create the New User
    newuser = gis.users.create(username = new_userid,
                                password = None,
                                firstname = olduser.firstName,
                                lastname = olduser.lastName,
                                email = olduser.email,
                                description = olduser.description,
                                role = 'org_user',
                                provider = 'enterprise',
                                idp_username=new_UPN)
else:
    # Based on the old user's info, create the New User
    newuser = gis.users.create(username = new_userid,
                                password = None,
                                firstname = olduser.firstName,
                                lastname = olduser.lastName,
                                email = olduser.email,
                                description = olduser.description,
                                role = olduser.role,
                                provider = 'enterprise',
                                idp_username=new_UPN)
print('Created User: {}'.format(new_userid))

#### Set the language of the new user to match the old user

In [ ]:
newuser.update(culture=olduser.culture)
print('Updated language to: {}'.format(newuser.culture))

#### Get the details for the New User

In [ ]:
newuser = gis.users.get(new_userid)
newuser

#### Reassign Groups

In [ ]:
usergroups = olduser['groups']
for group in usergroups:
    grp = gis.groups.get(group['id'])
    if (grp.owner == orig_userid):
        grp.reassign_to(new_userid)
    else:
        grp.add_users([new_userid])
        grp.remove_users([orig_userid])
        
print('Reassigned Groups')

#### Reassign Content

In [ ]:
usercontent = olduser.items()
folders = olduser.folders
for item in usercontent:
    try:
        item.reassign_to(new_userid)
    except Exception as e:
        print("Item may have been already assigned to the user.")

for folder in folders:
    gis.content.create_folder(folder['title'], new_userid)
    folderitems = olduser.items(folder=folder['title'])
    for item in folderitems:
        item.reassign_to(new_userid, target_folder=folder['title'])

print('Reassigned Content')

#### Disable Old User Account

In [ ]:
olduser.disable()
print('Disabled Old User')

#### Assign Entitlements

In [ ]:
if oldUserEntitlements:
    licenses.get(name="arcgis pro").assign(username=new_userid, entitlements=oldUserEntitlements, supress_email=True)
    print('Assigned entitlements')
else:
    print('User does not have entitlements')
if bool(licenses.get(name="arcgis pro").user_entitlement(username=new_userid)) == True:
    print('User has entitlements')